# SETUP CELLS

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
import requests
import json

# === Setup Paths ===
sys.path.insert(0, os.path.abspath("./genie-worksheets/src"))
root_dir = os.getcwd()

# === Load Environment ===
env_file = os.path.join(root_dir, ".env")
load_dotenv(env_file)

# === Verify API Key ===
AGENT_KEY = os.getenv("LLM_API_KEY")
BASE_URL = os.getenv("LLM_API_BASE_URL")
DATABASE_KEY = os.getenv("EXA_API_KEY")


if not AGENT_KEY or AGENT_KEY == "your_api_key_here":
    raise ValueError("❌ Please set your LLM_API_KEY in .env file")

print("✓ Environment loaded")
print(f"✓ API Base URL: {BASE_URL}")


# === Import only what you need from Genie ===
from worksheets.specification.from_spreadsheet import gsheet_to_classes
print("✓ Genie Worksheets imported")

✓ Environment loaded
✓ API Base URL: https://cs224v-litellm.genie.stanford.edu
✓ Genie Worksheets imported


In [2]:
# Ensure Environment is actually working
import requests
import json
from dotenv import load_dotenv

response = requests.post(
    f"{BASE_URL}/chat/completions",
    headers={
        "Authorization": f"Bearer {AGENT_KEY}",
        "Content-Type": "application/json"
    },
    json={
        "model": "gpt-4.1",
        "messages": [
            {"role": "user", "content": "Say hi in one word"}
        ],
        "max_tokens": 5
    }
)

if response.status_code == 200:
    result = response.json()
    print("Success!")
    print(f"Response: {result['choices'][0]['message']['content']}")
    print(f"Tokens used: {result.get('usage', {}).get('total_tokens', 'N/A')}")
else:
    print(f"Error {response.status_code}: {response.text}")

Success!
Response: Hello!
Tokens used: 15


In [3]:
current_dir = os.getcwd()
root_dir = os.path.join(current_dir, "genie-worksheets")

# Credentials for reading the spreadsheet
!curl -L -o creds.zip "https://drive.google.com/uc?export=download&id=11QvSs2JZ5qpPrCvbX66Dg8pxfM_B8GaH"
!unzip creds.zip -d genie-worksheets/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3009  100  3009    0     0   2069      0  0:00:01  0:00:01 --:--:--  419k
Archive:  creds.zip
  inflating: genie-worksheets/worksheets/token.json  
  inflating: genie-worksheets/worksheets/credentials.json  
  inflating: genie-worksheets/worksheets/service_account.json  


In [6]:
!mv "genie-worksheets/worksheets/credentials.json" "genie-worksheets/src/worksheets/config/credentials.json"
!mv "genie-worksheets/worksheets/service_account.json" "genie-worksheets/src/worksheets/config/service_account.json"
!mv "genie-worksheets/worksheets/token.json" "genie-worksheets/src/worksheets/config/token.json"

creds1 = os.path.join(root_dir, "src", "worksheets", "config", "credentials.json")
creds2 = os.path.join(root_dir, "src", "worksheets", "config", "service_account.json")
creds3 = os.path.join(root_dir, "src", "worksheets", "config", "token.json")

for cred in [creds1, creds2, creds3]:
  assert os.path.exists(cred), f"Cannot find the credential file: {cred}"

In [7]:
from worksheets.agent.config import agent_api

CELESTE_LEVELS = [
    {
        "level_id": "1",
        "room_id": "1",
        "difficulty": "easy",
        "objective": "reach the chute",
        "mechanics_involved": ["dash", "jump"],
        "hint_tiers": ["Try dashing right", "Climb the wall after dashing", "Use the platform timing"],
        "is_optional": False,
        "provenance_url": "https://celestegame.fandom.com/wiki/Forsaken_City",
        "side": "A"
    }
    # Will eventually be a full database, more will be added later.
]

@agent_api("get_hint", "Get a hint for a Celeste level")
def get_hint(
    level_id: str, 
    room_id: str, 
    hint_tier: int
):
    """
    Get a hint for a specific level.
    
    Args:
        level_id: The level ID
        room_id: The room ID
        hint_tier: Which hint to return (0, 1, or 2)
    """
    # Find the level
    level = next(
        (lvl for lvl in CELESTE_LEVELS 
         if lvl["level_id"] == level_id and lvl["room_id"] == room_id),
        None
    )
    
    if not level:
        return {
            "status": "error",
            "message": f"Level {level_id}, Room {room_id} not found"
        }
    
    # Validate hint_tier
    if hint_tier < 0 or hint_tier >= len(level["hint_tiers"]):
        return {
            "status": "error",
            "message": f"Hint tier {hint_tier} not available. Available: 0-{len(level['hint_tiers'])-1}"
        }
    
    return {
        "status": "success",
        "level_id": level_id,
        "room_id": room_id,
        "objective": level["objective"],
        "hint_tier": hint_tier,
        "hint": level["hint_tiers"][hint_tier]
    }